# MATH 189 Final Project

by: Pranav Damal, Lucien Chen, Venz Burgos

## Objective: ...

## Thesis: ...

In [2]:
# pip install nfl_data_py
import nfl_data_py as nfl
import pandas as pd
import numpy as np

## Top NFL QB per year 2006-2023

In [3]:
year = [2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
highestQBR = {}
for x in year:
    df = nfl.import_qbr([x])
    rank1 = df['rank'].idxmin()
    name = df.loc[rank1, 'name_display']
    highestQBR[x] = name

# highestQBR is a dict with the year and rank 1 QB
# incase this data is needed
print(highestQBR)

{2006: 'Peyton Manning', 2007: 'Tom Brady', 2008: 'Peyton Manning', 2009: 'Drew Brees', 2010: 'Tom Brady', 2011: 'Aaron Rodgers', 2012: 'Peyton Manning', 2013: 'Peyton Manning', 2014: 'Tony Romo', 2015: 'Carson Palmer', 2016: 'Matt Ryan', 2017: 'Carson Wentz', 2018: 'Patrick Mahomes', 2019: 'Lamar Jackson', 2020: 'Aaron Rodgers', 2021: 'Aaron Rodgers', 2022: 'Patrick Mahomes', 2023: 'Brock Purdy'}


## Top College QB Prospects 2006-2023

In [62]:
top_qb_prospects = {}

for year in range(2010, 2024):
    draft = nfl.import_draft_picks([year])
    firstQBs = draft[(draft['position'] == 'QB') & (draft['round'] == 1)]
    qbList = firstQBs['pfr_player_name'].tolist()
    top_qb_prospects[year] = qbList

for year, prospects in top_qb_prospects.items():
    print(f"first round QBs in {year}: {prospects}")

first round QBs in 2010: ['Sam Bradford', 'Tim Tebow']
first round QBs in 2011: ['Cam Newton', 'Jake Locker', 'Blaine Gabbert', 'Christian Ponder']
first round QBs in 2012: ['Andrew Luck', 'Robert Griffin III', 'Ryan Tannehill', 'Brandon Weeden']
first round QBs in 2013: ['EJ Manuel']
first round QBs in 2014: ['Blake Bortles', 'Johnny Manziel', 'Teddy Bridgewater']
first round QBs in 2015: ['Jameis Winston', 'Marcus Mariota']
first round QBs in 2016: ['Jared Goff', 'Carson Wentz', 'Paxton Lynch']
first round QBs in 2017: ['Mitchell Trubisky', 'Patrick Mahomes', 'Deshaun Watson']
first round QBs in 2018: ['Baker Mayfield', 'Sam Darnold', 'Josh Allen', 'Josh Rosen', 'Lamar Jackson']
first round QBs in 2019: ['Kyler Murray', 'Daniel Jones', 'Dwayne Haskins']
first round QBs in 2020: ['Joe Burrow', 'Tua Tagovailoa', 'Justin Herbert', 'Jordan Love']
first round QBs in 2021: ['Trevor Lawrence', 'Zach Wilson', 'Trey Lance', 'Justin Fields', 'Mac Jones']
first round QBs in 2022: ['Kenny Picket

## QB Data

In [18]:
QBCollege = pd.read_csv('./data/QBCollege.csv')
QBNFL = pd.read_csv('./data/QBNFL.csv')

QBCollege.head(5)

,player,year,player_id,position,team_name,player_game_count,accuracy_percent,aimed_passes,attempts,avg_depth_of_target,...,sack_percent,sacks,scrambles,spikes,thrown_aways,touchdowns,turnover_worthy_plays,twp_rate,yards,ypa
0,Michael Penix Jr.,2023,82134,QB,WASHINGTON,15,74.5,517,556,10.7,...,2.1,12,8,0,31,36,12.0,2.0,4906.0,8.8
1,Davis Brin,2023,77642,QB,GA SOUTHRN,13,75.0,525,556,8.4,...,4.9,30,21,5,15,24,31.0,4.9,3786.0,6.8
2,Brayden Schager,2023,146078,QB,HAWAII,13,72.9,495,526,10.3,...,6.7,40,32,2,17,26,14.0,2.2,3525.0,6.7
3,Alan Bowman,2023,77679,QB,OKLA STATE,14,71.8,457,502,8.6,...,1.2,6,5,0,42,15,22.0,4.1,3460.0,6.9
4,Cameron Ward,2023,133244,QB,WASH STATE,12,78.6,444,486,7.7,...,7.6,43,36,0,27,25,21.0,3.6,3723.0,7.7


In [14]:
QBNFL.head(5)

,Player,Year,Plays,EPA+CPOE composite,Adj. EPA/play,EPA/play,Success rate,Cmp%,Expected cmp%,CPOE,Air yards
0,S.Clifford,2023,2,0.581,0.648,0.648,50.00%,100.0,41.4,58.6,34.0
1,J.Brissett,2023,30,0.383,0.742,0.742,56.70%,81.8,64.2,17.7,8.0
2,B.Purdy,2023,646,0.188,0.333,0.316,54.30%,70.7,66.8,3.9,8.2
3,C.Wentz,2023,46,0.182,0.268,0.266,56.50%,73.9,67.2,6.7,6.7
4,C.Beathard,2023,68,0.162,0.171,0.171,51.50%,76.9,67.2,9.8,6.4
